# TechXchange Japan 2024: さわってみようベクトル・データベース watsonx.dataでRAG体験


生成AIの回答精度を向上させるために、自社内のデータを活用してみましょう！<br>
ベクトル・データベース + 大規模言語モデル（LLM）で構成されるRAGのアーキテクチャーを使えば、自社内の情報で生成AIチャットボットが作成できます。<br>
当ハンズオンでは「TechXchange Japan 2024」に関するデータを使ってRAGを構成し、「TechXchange Japan 2024」に関することを教えてくれるチャットボットを作成します。<br>
(以下ベクトル・データベースはベクトルDBと表記します。)<br>


具体的には大規模言語モデル（LLM）を使用したアプリケーション開発のためのオープンソース・オーケストレーション・フレームワーク[LangChain](https://python.langchain.com/docs/introduction/)を使って、wastosonx.dataのベクトルDB **Milvus**に「TechXchange Japan 2024」に関するデータをロードし、watson.aiで提供されているLLMを使用してRAGを構成し、「TechXchange Japan 2024」のことを回答してくれるChatbotを作ってみます。

ハンズオンは以下の順序で実行します:

1. Excelをベクトル化してベクトル・データベース Milvusに入れよう！
1. ベクトル・データベース Milvusに入ったデータで類似検索してみよう! 
1. ベクトル・データベース Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう! 
1. ベクトル・データベース Milvusとwatsonx.ai LLMでRAGを構成して、チャットアプリを作成してみよう!  [**当notebook**]


## 4.ベクトル・データベース Milvusとwatsonx.ai LLMでRAGを構成して、チャットアプリを作成してみよう! 

### 1. 必要なライブラリーのインストール

In [ ]:
!pip install -Uq 'ibm-watsonx-ai>=1.1.15'
!pip install -Uq 'langchain>=0.3.3'
!pip install -Uq 'langchain-ibm>=0.3.1'
!pip install -Uq 'langchain-huggingface>=0.1.0'
!pip install -Uq 'langchain-milvus>=0.1.6'
!pip install -Uq 'langchain-community>=0.3.2'
!pip install -Uq 'pymilvus>=2.4.8
!pip install -Uq gradio

In [ ]:
# エラーが出るので一応整合性確認
!pip check

`langchain-chroma`と`langchain-elasticsearch`は今回使用していないので問題ないです。<br>
**インストール終了後、一旦カーネルを再起動してください**

### 2. apikeyの設定 

- 事前に取得したapikeyを<api_key>　に入れる
    - 例:  `apikey="YyyyyyyyXxxxxxxxxxxxxZzzzzzzzzzzzzz"`


In [ ]:
apikey="<api_key>"

### 3. Milvus接続情報の設定

- watsonx.dataの画面を開く
- ナビゲーションメニューから「インフラストラクチャー・マネージャー」を選択
- サービス「Milvus」をクリック
- タイプの下の「接続の詳細を見る」をクリック
- GRPC ホストの値を<milvus GRPC ホスト>　に入れる
    - 例:  `milvus_host="xxxxxxxxxxx-xxxxxxxxxxx-xxxxxxxx.xxxxxxxx.lakehouse.appdomain.cloud"`
- GRPC ポートの値を<milvus GRPC ポート>　に入れる
    - 例:  `milvus_port="9999"`
      
collection名は`techxchange_line_data`としています。

In [2]:
milvus_host="<milvus GRPC ホスト>"
milvus_port="<milvus GRPC ポート>"

my_connection_args ={
 'uri': f'https://{milvus_host}:{milvus_port}', 
 'token': f'ibmlhapikey:{apikey}'
}
my_collection =  'techxchange_line_data'

### 3. watsonx.ai Project idの設定

Watson Studioで実行する場合は、このノートブックが実行されるプロジェクトからProject idを取得します。
Watson Studio以外で実行する場合は、Project idを入力してください。

**Hint**: `project_id` はプロジェクトを表示し、管理タブから `project_id` を取得可能です.

In [4]:
import os
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Project idを入力してください (入力後enter): ")

### 5.watsonx.aiのAuthentication用のエンドポイントのURLの設定

Waston Machine Learningのインスタンスを作成したリージョンで決まります。
https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication　より

- Dallas: https://us-south.ml.cloud.ibm.com
- London: https://eu-gb.ml.cloud.ibm.com
- Frankfurt: https://eu-de.ml.cloud.ibm.com
- Tokyo: https://jp-tok.ml.cloud.ibm.com

今回はダラスのWaston Machine Learningのインスタンスを使っているので`https://us-south.ml.cloud.ibm.com`を使います。

In [5]:
watsonx_url = "https://us-south.ml.cloud.ibm.com" #watsonx.aiのAuthentication用のエンドポイントのURL

### 4. 必要ライブラリーのImport

In [6]:
import pandas as pd
from langchain.schema.document import Document
import json
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
import os
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from langchain_ibm import WatsonxLLM

os.environ["TOKENIZERS_PARALLELISM"] = "false"

### 4. Embeddingモデルの取得
ベクトル化した時と同じモデル`intfloat/multilingual-e5-large`を使います<br>
https://huggingface.co/intfloat/multilingual-e5-large

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

### 5.  ベクトルDB Milvusに接続

In [8]:
from langchain_milvus import Milvus

vector_db = Milvus(
    embeddings,
    connection_args =my_connection_args,
    collection_name = my_collection
)

### 6.  watsonx.ai LLMの取得

In [9]:
# 使用するLLMのパラメータ
generate_params = {
    GenParams.MAX_NEW_TOKENS: 16384,
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1
}

# LangChainで使うllm
custom_llm = WatsonxLLM(
    # model_id="ibm/granite-3-8b-instruct", #使用するLLM名
    # model_id="meta-llama/llama-3-2-3b-instruct", #使用するLLM名
    model_id="mistralai/mixtral-8x7b-instruct-v01",
    url=watsonx_url,
    apikey=apikey,
    project_id=project_id,
    params=generate_params,
)


### 8. LangChainのchain作成

In [16]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# retrieverは単純なデフォルトのものにします（変えてもOK）
retriever = vector_db.as_retriever()

# Prompt Templateを設定します。これはmixtral-8x7b-instruct-v01に合わせて作成してあります
template = """<s> [INST] 
questionに答えるために、以下のcontextを使用し必ず日本語でanswerを作成してください。
必ず⽇本語の文章で回答してください。知ったかぶりをしないでください。
回答を書くときは、context内の単語をできるだけ使⽤してください。context中に質問に対する回答が無い低い場合は、「文書中に質問に対する回答が明記されていません。」とだけ回答してください。「文書中に質問に対する回答が明記されていません。」と回答した場合、そこで回答を終わりにしてください。
あなたは親切で、礼儀正しく、誠実なアシスタントです。常に安全を保ちながら、できるだけユーザーの役に立つように詳しく回答してください。
回答には、有害、非倫理的、⼈種差別的、性差別的、有毒、危険、または違法なコンテンツを含めてはいけません。回答は社会的に偏⾒がなく、本質的に前向きなものであることを確認してください。
質問が意味をなさない場合、または事実に⼀貫性がない場合は、正しくないことに答えるのではなく、その理由を説明してください。質問の答えがわからない場合は、誤った情報を共有しないでください。
セッションについて回答する場合は、タイトル、時間、会場、概要、レベルを回答してください。回答が200文字以上の場合、回答はなるべく箇条書きを含めてわかりやすく回答してください。
[/INST]
</s>
<s> [INST] 質問が質問の文章ではなく意味がわからない場合[/INST]もう少し詳しく説明していただけますか？</s>

context: {context}
question: {question}
answer: 
"""

# Prompt Templateを作成します
rag_prompt = PromptTemplate.from_template(template)

# chainを作成します
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | custom_llm
)

# 9. gradioでGUI作成(作成中)    

In [ ]:
import gradio as gr

def echo(message, history):
    print(json.dumps(history))

    return rag_chain.invoke(message)

demo = gr.ChatInterface(fn=echo, type="messages", examples=["hello", "hola", "merhaba"], title="Echo Bot")
demo.launch(share=True)

ベクトルDBに入れていない情報は回答しないようにプロンプトで指示しているので、回答しません。これで不正確な回答が防げます。<br>
ただしLLMによってそのような指示を書いても、無視するものもあったり、書き方で回答が変わったりするので、テストでプロンプトを調整したり、必要に応じて不要な回答をなくす後処理を追加してください。


---

これで「3. ベクトル・データベース Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう!」は完了です。<br>
次の「4. ベクトル・データベース Milvusとwatsonx.ai LLMでRAGを構成して、チャットアプリを作成してみよう!」に進んでください。